Downloading "War and Peace"

In [ ]:
import requests
url = "http://www.gutenberg.org/files/2600/2600-0.txt"
text = requests.get(url).content.decode()

In [ ]:
with open('war_and_peace.txt',mode='w',encoding='utf-8') as outputfile:
    outputfile.write(text)

In [ ]:
with open('war_and_peace.txt',mode='r',encoding='utf-8') as inputfile:
    text = inputfile.read()

In [ ]:
len(text)

In [ ]:
text[:1000]

In [ ]:
# skipping meta info and index at the beginning of the book
text = text[7285:]
print(text[:400])

In [ ]:
# removing CHAPTER heading and new lines
text = text.replace('CHAPTER','')
text = text.replace('\n',' ')
text = text.replace('\r',' ')

Tokenization

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk import word_tokenize
tokens = word_tokenize(text)

In [ ]:
len(tokens)

In [ ]:
print(tokens[2000:2050])

Sentence splitting

In [ ]:
from nltk import sent_tokenize

In [ ]:
sentences = sent_tokenize(text)

In [ ]:
len(sentences)

In [ ]:
print(sentences[6])

Let's tokenize every sentence.
This is the expected input for word2vec.

In [ ]:
tok_sents = [word_tokenize(sentence) for sentence in sentences]

In [ ]:
tok_sents[0:2]

A few statistics

In [ ]:
vocab = set(tokens)
len(vocab)

In [ ]:
from collections import Counter
frequencies = list(Counter(tokens).items())
frequencies[:30]

In [ ]:
frequencies.sort(key=lambda x: x[1], reverse=True)

In [ ]:
for word, frequency in frequencies[:30]:
    print(word,'\t',frequency)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.loglog([i+1 for i,_ in enumerate(frequencies)],[f for w,f in frequencies])
plt.show()

### Fitting a Word2Vec model

A tutorial from gensim's author is available at https://rare-technologies.com/word2vec-tutorial/ The tutorial contains details and links on how gensim implements word2vec.

This logging-related command enables printed feedback during the fitting process.

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

The creation of the Word2Vec object coincides with the fitting of the model.
It thus takes some time.

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(tok_sents, size=100, window=10, min_count=5, sg=1, iter=20, negative=10)

In [ ]:
w2v_model.wv.most_similar(['said'])

In [ ]:
w2v_model.wv.most_similar(['sir','she'],['he'])

In [ ]:
w2v_model.wv.vectors[1]

In [ ]:
w2v_model.wv.most_similar('father')

In [ ]:
import requests
url = "https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt"
test_file = 'questions-words.txt'
questions = requests.get(url).content.decode()
with open(test_file,mode='w',encoding='utf-8') as outputfile:
    outputfile.write(questions)
print(questions[:1000])

In [ ]:
w2v_wAp_analogy = w2v_model.wv.evaluate_word_analogies(test_file)

In [ ]:
w2v_wAp_analogy = w2v_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)

Same data, using a FastText model (sub-word character n-grams get their own embeddings).

In [ ]:
from gensim.models import FastText
fast_model = FastText(tok_sents,size=100, window=10, min_count=5, sg=1, iter=20, negative=10)

In [ ]:
fast_model.wv.most_similar(['said'])

In [ ]:
w2v_model.wv.most_similar('father')

In [ ]:
w2v_model.wv.most_similar('fathher')

In [ ]:
fast_model.wv.most_similar('father')

In [ ]:
fast_model.wv.most_similar('fathher')

In [ ]:
fasttext_wAp_analogy = fast_model.wv.evaluate_word_analogies(test_file)

In [ ]:
fasttext_wAp_analogy = fast_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)

Using google's precomputed word embeddings

This section works only after downloading and uncompressing Google's precomputed word embeddings into the same directory where this notebook is saved

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
from gensim.models import KeyedVectors
google_model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
google_model.most_similar(['king','female'],['male'])

In [ ]:
google_model.most_similar(['summer','cold'],['warm'])

In [ ]:
w2v_large_analogy = google_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)

In [ ]:
google_model.doesnt_match(['sun','moon','sand','jupiter'])

Using fasttext's precomputed word embeddings

https://fasttext.cc/docs/en/english-vectors.html

In [ ]:
del google_model

In [ ]:
!wget -c 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz'

In [ ]:
%ls

In [ ]:
!gunzip cc.en.300.bin.gz

In [ ]:
%ls

In [ ]:
from gensim.models.fasttext import FastText
fasttext_model = FastText.load_fasttext_format('cc.en.300.bin')

In [ ]:
fasttext_model.wv.most_similar('father')

In [ ]:
fasttext_model.wv.most_similar('fathher')

In [ ]:
fasttext_large_analogy = fasttext_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)